<a href="https://colab.research.google.com/github/hamisB/machine-learning/blob/master/TEDTalks_Recommender_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Data & Librairies


In [0]:
# import librairies 
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight") 
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import pandas_profiling as pp
import matplotlib.pyplot as plt

In [0]:
# Get data
data = pd.read_csv('https://storage.googleapis.com/source_market_fr/data_for_ml/ted_main.csv')
transcript = pd.read_csv('https://storage.googleapis.com/source_market_fr/data_for_ml/transcripts.csv')


In [0]:
# Fixing url as index and merge the two sub dataset
data.set_index('url', inplace=True)
transcript.set_index('url', inplace=True)

dataset = data.merge(transcript, right_index=True, left_index=True)

In [0]:
dataset_ml = dataset[['title','transcript']].reset_index().set_index('title')

In [42]:
# EDA
dataset_ml.drop('url', axis=1, inplace=True)
dataset_ml

,transcript
title,
"The mothers who found forgiveness, friendship",Phyllis Rodriguez: We are here today because o...
My year of living biblically,I thought I'd tell you a little about what I l...
A robot that flies like a bird,It is a dream of mankind to fly like a bird. B...
A TED speaker's worst nightmare,Today I'm going to talk about unexpected disco...
A whistleblower you haven't heard,(Whistling)(Whistling ends)(Applause)Thank you...
...,...
Machine intelligence makes human morals more important,"So, I started my first job as a computer progr..."
"My daughter, Malala",In many patriarchal societies and tribal socie...
The demise of guys?,"So today, I want us to reflect on the demise o..."


In [43]:
# Import NLTK and TfidVectorizer
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
# Processing function
STOPWORDS = set(nltk.corpus.stopwords.words('french'))
STEMMER = nltk.stem.SnowballStemmer('french')
# Cleaning function
def clean_text(text):
    text_lower = text.lower()
    text_clean = re.sub('[^a-z_]', ' ', text_lower)
    tokens = [w for w in text_clean.split() if (len(w)>2) and (w not in STOPWORDS)]
    tokens_stem = [ STEMMER.stem(token) for token in tokens]
    return tokens_stem

In [0]:
# Create a vectorizer with 100 features 
vectorizer = TfidfVectorizer( tokenizer=clean_text , max_features=10000, ngram_range=(1,2))

In [48]:
# Create the Tdif matrix and check the feature
df_tdif = vectorizer.fit_transform(dataset_ml.transcript)
vectorizer.get_feature_names()[:25]

['abandoned',
 'abilit',
 'ability',
 'able',
 'able get',
 'able mak',
 'able se',
 'about',
 'about all',
 'about and',
 'about being',
 'about billion',
 'about but',
 'about dollar',
 'about fiv',
 'about four',
 'about half',
 'about her',
 'about his',
 'about how',
 'about million',
 'about minut',
 'about one',
 'about other',
 'about our']

In [0]:
# Calcul similarity 
from sklearn.metrics.pairwise import linear_kernel
similarity = linear_kernel(df_tdif, df_tdif)

In [0]:
sim_score = pd.DataFrame(similarity, index=dataset_ml.index , columns=dataset_ml.index)

In [55]:
sim_score["Superheroes inspired by Islam"].sort_values(ascending=False)[1:11]

title
The web is more than "better TV"                           0.723792
Dreams from endangered cultures                            0.721669
The worldwide web of belief and ritual                     0.710123
My wish: Manufactured landscapes and green education       0.702625
Design and discovery                                       0.701913
My days as a young rebel                                   0.697814
The unheard story of the Sistine Chapel                    0.696906
Nationalism vs. globalism: the new political divide        0.696232
Rebuilding a neighborhood with beauty, dignity, hope       0.692869
Spider-Man, The Lion King and life on the creative edge    0.687954
Name: Superheroes inspired by Islam, dtype: float64

In [56]:
sim_score["My days as a young rebel"].sort_values(ascending=False)[1:11]

title
Building uniqueness                                        0.755281
How architecture can connect us                            0.754915
Rebuilding a neighborhood with beauty, dignity, hope       0.754517
My green agenda for architecture                           0.749676
My wish: Manufactured landscapes and green education       0.740199
The Blur Building and other tech-empowered architecture    0.730747
Building the Seed Cathedral                                0.730361
Design and discovery                                       0.725368
The web is more than "better TV"                           0.724961
A memorial at Ground Zero                                  0.719566
Name: My days as a young rebel, dtype: float64